In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import dlib

# Specify the paths to the Haar cascade files
face_cascade_path = 'haarcascade_frontalface_default.xml'
eye_cascade_path = 'haarcascade_eye.xml'
profile_cascade_path = 'haarcascade_profileface.xml'

# Load the cascades
face_cascade = cv2.CascadeClassifier(face_cascade_path)
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)
profile_cascade = cv2.CascadeClassifier(profile_cascade_path)

# Function to detect faces (both frontal and profile) and eyes
def detect_faces_and_eyes(gray):
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    if len(faces) == 0:
        faces = profile_cascade.detectMultiScale(gray, 1.1, 4)
    eyes_detected = []
    for (x, y, w, h) in faces:
        face_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(face_gray)
        eyes_detected.append((x, y, w, h, eyes))
    return faces, eyes_detected

# Function to normalize and enhance contrast of face images
def normalize_and_enhance(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces, eyes_detected = detect_faces_and_eyes(gray)
    for (x, y, w, h, eyes) in eyes_detected:
        face_gray = gray[y:y+h, x:x+w]
        if len(eyes) >= 2:
            eye_1, eye_2 = eyes[:2]
            eye_center_1 = (eye_1[0] + eye_1[2] // 2, eye_1[1] + eye_1[3] // 2)
            eye_center_2 = (eye_2[0] + eye_2[2] // 2, eye_2[1] + eye_2[3] // 2)
            if eye_center_1[0] > eye_center_2[0]:
                eye_center_1, eye_center_2 = eye_center_2, eye_center_1
            dY = eye_center_2[1] - eye_center_1[1]
            dX = eye_center_2[0] - eye_center_1[0]
            angle = np.degrees(np.arctan2(dY, dX))
            center_of_face = (int(w // 2), int(h // 2))
            M = cv2.getRotationMatrix2D(center_of_face, angle, 1)
            rotated = cv2.warpAffine(face_gray, M, (w, h))
            equalized = cv2.equalizeHist(rotated)
            resized = cv2.resize(equalized, (70, 70))
            return resized
    return None

# Path to the dataset folder
dataset_path = './training-originals'  # Update with the correct path to your dataset folder

# List all image files in the dataset folder
image_files = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) if f.endswith('.jpg')]

# Normalize and enhance images in the dataset
normalized_faces = []
face_labels = []
for image_file in image_files:
    normalized_face = normalize_and_enhance(image_file)
    if normalized_face is not None:
        normalized_faces.append(normalized_face)
        # Extract the label from the filename
        label = os.path.basename(image_file).split('_')[0]
        face_labels.append(label)
        print(f"Processed {image_file}, label: {label}")

print(f"Normalized and enhanced {len(normalized_faces)} face images.")

# Display a few sample images after normalization and enhancement
plt.figure(figsize=(10, 5))
for i in range(min(5, len(normalized_faces))):
    plt.subplot(1, 5, i+1)
    plt.imshow(normalized_faces[i], cmap='gray')
    plt.title(face_labels[i])
    plt.axis('off')
plt.show()

# Flatten normalized face images for PCA
data = np.array([face.flatten() for face in normalized_faces])

# Fit PCA with adjusted number of components
pca = PCA(n_components=10, whiten=True).fit(data)

# Project face images to PCA space
X_pca = pca.transform(data)

print("PCA-transformed data:")
print(X_pca[:5])  # Print first 5 PCA-transformed vectors

# Train a classifier (SVM) on the PCA-transformed data
scaler = StandardScaler()
X_pca_scaled = scaler.fit_transform(X_pca)
clf = SVC(probability=True)
clf.fit(X_pca_scaled, face_labels)

print("SVM classifier predictions on training data:")
train_predictions = clf.predict(X_pca_scaled)
for i in range(min(5, len(train_predictions))):
    print(f"Image: {face_labels[i]}, Predicted: {train_predictions[i]}")

print(f"Projected face images to {X_pca.shape[1]}-dimensional PCA space.")

# Function to recognize faces in webcam feed
def recognize_face_enhanced(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces, eyes_detected = detect_faces_and_eyes(gray)
    for (x, y, w, h, eyes) in eyes_detected:
        face_gray = gray[y:y+h, x:x+w]
        if len(eyes) >= 2:
            eye_1, eye_2 = eyes[:2]
            eye_center_1 = (eye_1[0] + eye_1[2] // 2, eye_1[1] + eye_1[3] // 2)
            eye_center_2 = (eye_2[0] + eye_2[2] // 2, eye_2[1] + eye_2[3] // 2)
            if eye_center_1[0] > eye_center_2[0]:
                eye_center_1, eye_center_2 = eye_center_2, eye_center_1
            dY = eye_center_2[1] - eye_center_1[1]
            dX = eye_center_2[0] - eye_center_1[0]
            angle = np.degrees(np.arctan2(dY, dX))
            center_of_face = (int(w // 2), int(h // 2))
            M = cv2.getRotationMatrix2D(center_of_face, angle, 1)
            rotated = cv2.warpAffine(face_gray, M, (w, h))
            equalized = cv2.equalizeHist(rotated)
            face_resized = cv2.resize(equalized, (70, 70)).flatten()
            face_pca = pca.transform([face_resized])
            face_pca_scaled = scaler.transform(face_pca)
            probabilities = clf.predict_proba(face_pca_scaled)
            max_prob = np.max(probabilities)
            label = clf.predict(face_pca_scaled)[0]
            if max_prob < 0.5:
                label = 'Unknown'
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, f"{label} ({max_prob:.2f})", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(frame, (x+ex, y+ey), (x+ex+ew, y+ey+eh), (0, 255, 0), 2)
    return frame

# Capture video from webcam
cap = cv2.VideoCapture(1, cv2.CAP_DSHOW)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)

    result_frame = recognize_face_enhanced(frame)
    cv2.imshow('Face Recognition', result_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
